In [18]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import lxml

req = requests.get("https://www.espncricinfo.com/records/season/team-match-results/2024-2024?trophy=89")
soup = bs(req.content)

In [19]:
summary_table = soup.find('table')
summary_df = pd.read_html(str(summary_table))[0]

C:\Users\swara\AppData\Local\Temp\ipykernel_32508\2996156663.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  summary_df = pd.read_html(str(summary_table))[0]


In [ ]:
summary_df['match_id'] = range(1, 53)
summary_df

summary_df.to_csv('summary.csv', index=False)

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Scorecard,match_id
0,India,South Africa,India,7 runs,Bridgetown,"Jun 29, 2024",T20I # 2729,1
1,England,India,India,68 runs,Providence,"Jun 27, 2024",T20I # 2724,2
2,Afghanistan,South Africa,South Africa,9 wickets,Tarouba,"Jun 26, 2024",T20I # 2723,3
3,Afghanistan,Bangladesh,Afghanistan,8 runs,Kingstown,"Jun 24, 2024",T20I # 2722,4
4,Australia,India,India,24 runs,Gros Islet,"Jun 24, 2024",T20I # 2721,5
5,West Indies,South Africa,South Africa,3 wickets,North Sound,"Jun 23, 2024",T20I # 2720,6
6,England,U.S.A.,England,10 wickets,Bridgetown,"Jun 23, 2024",T20I # 2719,7
7,Afghanistan,Australia,Afghanistan,21 runs,Kingstown,"Jun 22, 2024",T20I # 2717,8
8,Bangladesh,India,India,50 runs,North Sound,"Jun 22, 2024",T20I # 2716,9
9,West Indies,U.S.A.,West Indies,9 wickets,Bridgetown,"Jun 21, 2024",T20I # 2713,10


# Getting Individual Matches

In [3]:
# Creating an array containing the links of scorecards of all matches
match_links = summary_table.find_all('a',)

href_list = []
for link in match_links:
  href = link.get('href')
  href_list.append(href)

links = href_list[1::2]

# First Match Scorecard:

In [4]:
link = "https://espncricinfo.com" + links[0]
req = requests.get(link)
soup = bs(req.content)

## Applying For All Matches:

In [5]:
# Function to create a dataframe from batting table

def create_batting_df(batting_table, soup, count):
  columns = batting_table.find('thead').find_all('th')
  column_names = [c.string for c in columns]
  table_rows = batting_table.find('tbody').find_all('tr')
  l = []
  for tr in table_rows:
    td = tr.find_all('td')
    row = [str(tr.get_text()).strip() for tr in td]
    l.append(row[:-1]) # Exclude the last element of each row
  
  df = pd.DataFrame(l, columns=column_names[:-1])  # Exclude the last column name
  df.dropna(inplace=True)
  df = df.rename(columns={df.columns[1]: 'Dissmissal'})

  # Extracting the match name
  name = soup.select('h1.ds-text-title-xs.ds-font-bold.ds-mb-2.ds-m-1')
  match_name = name[0].get_text()
  match_name = match_name.split(',')[0] 
  df['Match'] = match_name

  # Extracting the batting team name
  innings1 = soup.select('span.ds-text-title-xs.ds-font-bold.ds-capitalize')[0].get_text()
  df['innings'] = innings1

  df['match_id'] = count
  
  return df

In [6]:
score_links = ["https://espncricinfo.com" + link for link in links]
df_batting = pd.DataFrame()
count = 1
for link in score_links:
  req = requests.get(link)
  soup = bs(req.content)
  batting_table = soup.find_all('table', class_="ci-scorecard-table")

  batting_first = batting_table[0]
  df1 = create_batting_df(batting_first, soup, count)

  if len(batting_table) == 2:
    batting_second = batting_table[1]
    df2 = create_batting_df(batting_second, soup, count)
    df_batting = pd.concat([df_batting, df1, df2], ignore_index=True)

  else:
    df_batting = pd.concat([df_batting, df1], ignore_index=True)

  count += 1

df_batting = df_batting.drop(columns=['SR'])


## For Bowling Scorecard:

In [7]:
link = score_links[0]
req = requests.get(link)
soup = bs(req.content)

link

'https://espncricinfo.com/series/icc-men-s-t20-world-cup-2024-1411166/india-vs-south-africa-final-1415755/full-scorecard'

In [8]:
bowling_table = soup.select('table.ds-w-full.ds-table.ds-table-md.ds-table-auto')
bowling_first = bowling_table[1]
bowling_second = bowling_table[3]

In [9]:
# Function to convert bowling table to dataframe

def create_bowling_df(bowling_table, soup, count):
  columns = bowling_table.find('thead').find_all('th')
  column_names = [c.string for c in columns]
  table_rows = bowling_table.find('tbody').find_all('tr')
  l = []
  for tr in table_rows:
    td = tr.find_all('td')
    row = [str(tr.get_text()).strip() for tr in td]

    if len(row) != len(column_names):
      row = row[:-1]

    l.append(row)

  df = pd.DataFrame(l, columns=column_names)
  df.dropna(inplace=True)

  # Extracting the match name
  name = soup.select('h1.ds-text-title-xs.ds-font-bold.ds-mb-2.ds-m-1')
  match_name = name[0].get_text()
  match_name = match_name.split(',')[0] 
  df['Match'] = match_name

  # Extracting the batting team name
  innings1 = soup.select('span.ds-text-title-xs.ds-font-bold.ds-capitalize')[0].get_text()
  df['innings'] = innings1

  df['match_id'] = count
  
  return df



In [10]:
req = requests.get(link)
soup = bs(req.content)
bowling_table = soup.select('table.ds-w-full.ds-table.ds-table-md.ds-table-auto')

bowling_first = bowling_table[1]
bowling_second = bowling_table[3]

columns = bowling_first.find('thead').find_all('th')
column_names = [c.string for c in columns]
table_rows = bowling_first.find('tbody').find_all('tr')
l = []
for tr in table_rows:
  td = tr.find_all('td')
  row = [str(tr.get_text()).strip() for tr in td]
  
  if len(row) != len(column_names):
    column_names.insert(1, np.nan)

  l.append(row)

In [11]:
df_bowling = pd.DataFrame()
count = 1

for link in score_links:
  req = requests.get(link)
  soup = bs(req.content)
  bowling_table = soup.select('table.ds-w-full.ds-table.ds-table-md.ds-table-auto')

  bowling_first = bowling_table[1]
  df1 = create_bowling_df(bowling_first, soup, count)


  if len(bowling_table) == 4:
    bowling_second = bowling_table[3]
    df2 = create_bowling_df(bowling_second, soup, count)
    df_bowling = pd.concat([df_bowling, df1, df2], ignore_index=True)

  else:
    df_bowling = pd.concat([df_bowling, df1], ignore_index=True)

  count += 1

In [14]:
df_batting

,Batting,Dissmissal,R,B,M,4s,6s,Match,innings,match_id
0,Rohit Sharma (c),c Klaasen b Maharaj,9,5,7,2,0,India vs South Africa,India,1
1,Virat Kohli,c Rabada b Jansen,76,59,87,6,2,India vs South Africa,India,1
2,Rishabh Pant †,c †de Kock b Maharaj,0,2,2,0,0,India vs South Africa,India,1
3,Suryakumar Yadav,c Klaasen b Rabada,3,4,9,0,0,India vs South Africa,India,1
4,Axar Patel,run out (†de Kock),47,31,38,1,4,India vs South Africa,India,1
...,...,...,...,...,...,...,...,...,...,...
847,Steven Taylor,lbw b Kaleem Sana,0,2,2,0,0,U.S.A. vs Canada,Canada,52
848,Monank Patel (c)†,c †Movva b Heyliger,16,16,33,2,0,U.S.A. vs Canada,Canada,52
849,Andries Gous,c Johnson b Dutta,65,46,76,7,3,U.S.A. vs Canada,Canada,52
850,Aaron Jones,not out,94,40,56,4,10,U.S.A. vs Canada,Canada,52


In [13]:
df_bowling.to_csv("bowling.csv", index=False)
df_batting.to_csv("batting.csv", index=False)